In [1]:
# !wget https://raw.githubusercontent.com/attardi/wikiextractor/master/WikiExtractor.py
# !wget http://dumps.wikimedia.org/mswiki/latest/mswiki-latest-pages-articles.xml.bz2
# !bzip2 -d mswiki-latest-pages-articles.xml.bz2
# !python3 WikiExtractor.py --json mswiki-latest-pages-articles.xml

In [2]:
from glob import glob

files = glob('text/*/*')
len(files)

256

In [3]:
import json

with open(files[0]) as fopen:
    x = fopen.readlines()

In [4]:
import re
from unidecode import unidecode

alphabets = '([A-Za-z])'
prefixes = (
    '(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt|Puan|puan|Tuan|tuan|sir|Sir)[.]'
)
suffixes = '(Inc|Ltd|Jr|Sr|Co)'
starters = '(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever|Dia|Mereka|Tetapi|Kita|Itu|Ini|Dan|Kami)'
acronyms = '([A-Z][.][A-Z][.](?:[A-Z][.])?)'
websites = '[.](com|net|org|io|gov|me|edu|my)'
another_websites = '(www|http|https)[.]'
digits = '([0-9])'

def cleaning(string):
    string = unidecode(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

def split_into_sentences(text):
    text = unidecode(text)
    text = ' ' + text + '  '
    text = text.replace('\n', ' ')
    text = re.sub(prefixes, '\\1<prd>', text)
    text = re.sub(websites, '<prd>\\1', text)
    text = re.sub(another_websites, '\\1<prd>', text)
    if '...' in text:
        text = text.replace('...', '<prd><prd><prd>')
    if 'Ph.D' in text:
        text = text.replace('Ph.D.', 'Ph<prd>D<prd>')
    text = re.sub('\s' + alphabets + '[.] ', ' \\1<prd> ', text)
    text = re.sub(acronyms + ' ' + starters, '\\1<stop> \\2', text)
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]' + alphabets + '[.]',
        '\\1<prd>\\2<prd>\\3<prd>',
        text,
    )
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]', '\\1<prd>\\2<prd>', text
    )
    text = re.sub(' ' + suffixes + '[.] ' + starters, ' \\1<stop> \\2', text)
    text = re.sub(' ' + suffixes + '[.]', ' \\1<prd>', text)
    text = re.sub(' ' + alphabets + '[.]', ' \\1<prd>', text)
    text = re.sub(digits + '[.]' + digits, '\\1<prd>\\2', text)
    if '”' in text:
        text = text.replace('.”', '”.')
    if '"' in text:
        text = text.replace('."', '".')
    if '!' in text:
        text = text.replace('!"', '"!')
    if '?' in text:
        text = text.replace('?"', '"?')
    text = text.replace('.', '.<stop>')
    text = text.replace('?', '?<stop>')
    text = text.replace('!', '!<stop>')
    text = text.replace('<prd>', '.')
    sentences = text.split('<stop>')
    sentences = sentences[:-1]
    sentences = [cleaning(s.strip()) for s in sentences if len(s) > 10]
    return sentences

In [5]:
from tqdm import tqdm

texts = []

for i in tqdm(range(len(files))):
    with open(files[i]) as fopen:
        s = fopen.readlines()
    
    for i in s:
        i = json.loads(i)
        texts.extend(split_into_sentences(i['text'].replace('<br>', ' ')))
        texts.append('')

100%|██████████| 256/256 [01:55<00:00,  2.22it/s]


In [6]:
with open('dumping-wiki.txt', 'w') as fopen:
    fopen.write('\n'.join(texts))

In [2]:
!head -n 100 wiki-text.txt

Alexandre Pato Alexandre Rodrigues da Silva (lahir 2 September, 1989, di Pato Branco, Parana), lebih dikenali sebagai Alexandre Pato, merupakan pemain bola sepak Brazil yang bermain sebagai penyerang untuk kelab Sepanyol, Villarreal.
Alexandre mula bermain futsal di usia 4 tahun.
Kebolehannya kemudian diketahui di seluruh kawasan selatan Brazilian iaitu di Parana.
Setelah menerima pujian oleh ramai jurulatihnya, remaja tersebut telah pergi ke Porto Alegre, Rio Grande do Sul, untuk mencuba nasib bersama Internacional.
Pada 2001, dalam usia 11 tahun, beliau berpindah ke sana dan mula berkongsi rumah dengan 83 orang remaja lain, yang juga sepertinya, ingin mencipta nama dalam skuad utama Internacional.
Pada 2000, sewaktu Alexandre berusia 10 tahun, x-ray dari tulang yang patah telah mendedahkan bahawa beliau mempunyai ketumbuhan pada lengannya.
Para doktor menyedari bahawa ketumbuhan itu boleh menyebabkan kanser dalam dua bulan sekiranya tidak dibuang.
Keluarga Alexandre tidak mampu menam